### Setup

apt install requirements

In [1]:
# !apt-get install -y \
#     libgl1-mesa-dev \
#     libgl1-mesa-glx \
#     libglew-dev \
#     libosmesa6-dev \
#     software-properties-common

# !apt-get install -y patchelf

Clone repo and install dependencies

In [2]:
# %cd /content

# ## TODO: use when repo is public
# # !git clone https://github.com/jannerm/diffuser.git
# ## backup for private repo
# !curl -L -o diffuser.zip https://www.dropbox.com/s/fe1scd8m2jf7zvj/diffuser.tar?dl=1
# !tar -xf diffuser.zip
# ## /TODO

# %cd diffuser
# %pip install -f https://download.pytorch.org/whl/torch_stable.html \
#                 typed-argument-parser \
#                 scikit-image==0.17.2 \
#                 scikit-video==1.1.11 \
#                 gitpython \
#                 einops \
#                 pillow \
#                 free-mujoco-py \
#                 gym \
#                 git+https://github.com/JannerM/d4rl.git@0e84a4d29db3ae501043215ce1d91843929f1949 \
#                 git+https://github.com/aravindr93/mjrl

# %pip install -e .

In [3]:
import os
import json
import numpy as np
from os.path import join
import pdb

from diffuser.guides.policies import Policy
import diffuser.datasets as datasets
import diffuser.utils as utils

/opt/conda/envs/diffuser/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
pybullet build time: Nov 28 2023 23:51:11


In [4]:
os.chdir('/root/diffuser')

Arguments

In [5]:
savepath = 'logs/maze2d-large-v1/rollouts'
os.makedirs(savepath, exist_ok=True)
vis_freq = 10
batch_size = 1

Loading

In [6]:
dataset = 'maze2d-large-v1'

env = datasets.load_environment(dataset)

diffusion_experiment = utils.load_diffusion('logs_2', dataset, 'diffusion/H384_T256', epoch='latest')

diffusion = diffusion_experiment.ema
dataset = diffusion_experiment.dataset
renderer = diffusion_experiment.renderer

policy = Policy(diffusion, dataset.normalizer)

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/render_config.pkl

[utils/config ] Config: <class 'diffuser.utils.rendering.Maze2dRenderer'>
    env: maze2d-large-v1

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/model_config.pkl

[utils/config ] Config: <class 'diffuser.models.temporal.TemporalUnet'>
    cond_dim: 4
    dim_mults: (1, 4, 8)
    horizon: 384
    transition_dim: 6

[ utils/serialization ] Loaded config from logs_2/maze2d-large-v1/diffusion/H384_T256/diffusion_config.pkl

[utils/config ] Config: <class 'diffuser.models.diffusion.GaussianDiffusion'>
  

load datafile: 100%|██████████| 8/8 [00:01<00:00,  7.16it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)
[ models/temporal ] Channel dimensions: [(6, 32), (32, 128), (128, 256)]
[(6, 32), (32, 128), (128, 256)]

[ utils/serialization ] Loading model epoch: 1960000



In [7]:
dataset.normalizer

actions: [ Normalizer ] dim: 2
    -: [-1. -1.]
    +: [1. 1.]
]
infos/goal: [ Normalizer ] dim: 2
    -: [0.9 0.9]
    +: [ 7.1 10.1]
]
infos/qpos: [ Normalizer ] dim: 2
    -: [0.4  0.44]
    +: [ 7.22 10.22]
]
infos/qvel: [ Normalizer ] dim: 2
    -: [-5.23 -5.23]
    +: [5.23 5.23]
]
observations: [ Normalizer ] dim: 4
    -: [ 0.4   0.44 -5.23 -5.23]
    +: [ 7.22 10.22  5.23  5.23]
]
rewards: [ Normalizer ] dim: 1
    -: [0.]
    +: [1.]
]
terminals: [ Normalizer ] dim: 1
    -: [0.]
    +: [0.]
]
timeouts: [ Normalizer ] dim: 1
    -: [0.]
    +: [0.]
]
next_observations: [ Normalizer ] dim: 4
    -: [ 0.4   0.44 -5.23 -5.23]
    +: [ 7.22 10.22  5.23  5.23]
]

Main Loop

In [8]:
from tqdm import tqdm

In [11]:
observation = env.reset()
isconditioned = True

if isconditioned:
    print('Resetting target')
    env.set_target()

## set conditioning xy position to be the goal
target = env._target
cond = {
    diffusion.horizon - 1: np.array([*target, 0, 0]),
}

## observations for rendering
rollout = [observation.copy()]

total_reward = 0
scorelist = []
for i in tqdm(range(1)):
    total_reward = 0
    for t in range(env.max_episode_steps):

        state = env.state_vector().copy()

        ## can replan if desired, but the open-loop plans are good enough for maze2d
        ## that we really only need to plan once
        if t == 0:
            cond[0] = observation

            action, samples = policy(cond, batch_size=batch_size)
            actions = samples.actions[0]
            sequence = samples.observations[0]
        # pdb.set_trace()

        # ####
        if t < len(sequence) - 1:
            next_waypoint = sequence[t+1]
        else:
            next_waypoint = sequence[-1].copy()
            next_waypoint[2:] = 0
            # pdb.set_trace()

        ## can use actions or define a simple controller based on state predictions
        action = next_waypoint[:2] - state[:2] + (next_waypoint[2:] - state[2:])
        # pdb.set_trace()
        ####

        # else:
        #     actions = actions[1:]
        #     if len(actions) > 1:
        #         action = actions[0]
        #     else:
        #         # action = np.zeros(2)
        #         action = -state[2:]
        #         pdb.set_trace()



        next_observation, reward, terminal, _ = env.step(action)
        total_reward += reward
        score = env.get_normalized_score(total_reward)
        print(
            f't: {t} | r: {reward:.2f} |  R: {total_reward:.2f} | score: {score:.4f} | ref_max_score: {env.ref_max_score} | ref_min_score: {env.ref_min_score}'
            f'{action}'
        )

        xy = next_observation[:2]
        goal = env.unwrapped._target
        print(f'maze | pos: {xy} | goal: {goal}')

        ## update rollout observations
        rollout.append(next_observation.copy())

        # logger.log(score=score, step=t)

        if t % vis_freq == 0 or terminal:
            fullpath = join(savepath, f'{t}.png')

            if t == 0: renderer.composite(fullpath, samples.observations, ncol=1)



            ## save rollout thus far
            renderer.composite(join(savepath, 'rollout.png'), np.array(rollout)[None], ncol=1)



        if terminal:
            break

        observation = next_observation
    scorelist.append((total_reward, score)) 

Resetting target


  0%|          | 0/1 [00:00<?, ?it/s]


                                                                                                    
1 / 256 [                                                            ]   0% | 95.0 Hz
t : 255
                                                                                                    
                                                                                                    
2 / 256 [                                                            ]   0% | 107.5 Hz
t : 254
                                                                                                    
                                                                                                    
3 / 256 [                                                            ]   1% | 112.7 Hz
t : 253
                                                                                                    
                                                                                                    
4 / 256 

--- Logging error in Loguru Handler #0 ---
Record was: {'elapsed': datetime.timedelta(seconds=69, microseconds=212042), 'exception': None, 'extra': {}, 'file': (name='maze_model.py', path='/opt/conda/envs/diffuser/lib/python3.8/site-packages/d4rl/pointmaze/maze_model.py'), 'function': 'step', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 197, 'message': 'sparse, mess it up', 'module': 'maze_model', 'name': 'd4rl.pointmaze.maze_model', 'process': (id=68604, name='MainProcess'), 'thread': (id=140349938877632, name='MainThread'), 'time': datetime(2024, 5, 30, 22, 28, 36, 560115, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))}
Traceback (most recent call last):
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_handler.py", line 206, in emit
    self._sink.write(str_record)
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_simple_sinks.py", line 16, in write
    self._stream.write(message)
ValueError: I/O operation on closed file.
--- End o

t: 1 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[0.36041867 0.03416838]
maze | pos: [1.98984169 3.97785546] | goal: [ 7.09047511 10.05128897]
t: 2 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 0.43758139 -0.05085708]
maze | pos: [1.99219704 3.97742667] | goal: [ 7.09047511 10.05128897]
t: 3 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 0.58606992 -0.05542276]
maze | pos: [1.99594258 3.97686691] | goal: [ 7.09047511 10.05128897]
t: 4 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[0.67579616 0.1180208 ]
maze | pos: [2.00128871 3.97658956] | goal: [ 7.09047511 10.05128897]
t: 5 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[0.63668506 0.02758349]
maze | pos: [2.00813846 3.97637856] | goal: [ 7.09047511 10.05128897]
t: 6 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[0.7

...

Saved 1 samples to: logs/maze2d-large-v1/rollouts/rollout.png
t: 741 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -3.41922607e-04]
maze | pos: [ 5.19850204 10.04136505] | goal: [ 7.09047511 10.05128897]
t: 742 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -3.38422976e-04]
maze | pos: [ 5.19850204 10.04146663] | goal: [ 7.09047511 10.05128897]
t: 743 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -3.34959165e-04]
maze | pos: [ 5.19850204 10.04156717] | goal: [ 7.09047511 10.05128897]
t: 744 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -3.31530806e-04]
maze | pos: [ 5.19850204 10.04166668] | goal: [ 7.09047511 10.05128897]
t: 745 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -3.28137536e-04]
maze | pos: [ 5.19850204 10.0417651

--- Logging error in Loguru Handler #0 ---
Record was: {'elapsed': datetime.timedelta(seconds=77, microseconds=97151), 'exception': None, 'extra': {}, 'file': (name='maze_model.py', path='/opt/conda/envs/diffuser/lib/python3.8/site-packages/d4rl/pointmaze/maze_model.py'), 'function': 'step', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 197, 'message': 'sparse, mess it up', 'module': 'maze_model', 'name': 'd4rl.pointmaze.maze_model', 'process': (id=68604, name='MainProcess'), 'thread': (id=140349938877632, name='MainThread'), 'time': datetime(2024, 5, 30, 22, 28, 44, 445224, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))}
Traceback (most recent call last):
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_handler.py", line 206, in emit
    self._sink.write(str_record)
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_simple_sinks.py", line 16, in write
    self._stream.write(message)
ValueError: I/O operation on closed file.
--- End of

Saved 1 samples to: logs/maze2d-large-v1/rollouts/rollout.png
t: 761 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.78335305e-04]
maze | pos: [ 5.19850204 10.04321071] | goal: [ 7.09047511 10.05128897]
t: 762 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.75486500e-04]
maze | pos: [ 5.19850204 10.0432934 ] | goal: [ 7.09047511 10.05128897]
t: 763 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.72666853e-04]
maze | pos: [ 5.19850204 10.04337524] | goal: [ 7.09047511 10.05128897]
t: 764 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.69876065e-04]
maze | pos: [ 5.19850204 10.04345624] | goal: [ 7.09047511 10.05128897]
t: 765 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.67113842e-04]
maze | pos: [ 5.19850204 10.0435364

--- Logging error in Loguru Handler #0 ---
Record was: {'elapsed': datetime.timedelta(seconds=77, microseconds=328754), 'exception': None, 'extra': {}, 'file': (name='maze_model.py', path='/opt/conda/envs/diffuser/lib/python3.8/site-packages/d4rl/pointmaze/maze_model.py'), 'function': 'step', 'level': (name='INFO', no=20, icon='ℹ️'), 'line': 197, 'message': 'sparse, mess it up', 'module': 'maze_model', 'name': 'd4rl.pointmaze.maze_model', 'process': (id=68604, name='MainProcess'), 'thread': (id=140349938877632, name='MainThread'), 'time': datetime(2024, 5, 30, 22, 28, 44, 676827, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))}
Traceback (most recent call last):
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_handler.py", line 206, in emit
    self._sink.write(str_record)
  File "/opt/conda/envs/diffuser/lib/python3.8/site-packages/loguru/_simple_sinks.py", line 16, in write
    self._stream.write(message)
ValueError: I/O operation on closed file.
--- End o

Saved 1 samples to: logs/maze2d-large-v1/rollouts/rollout.png
t: 781 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.26573325e-04]
maze | pos: [ 5.19850204 10.04471313] | goal: [ 7.09047511 10.05128897]
t: 782 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.24254312e-04]
maze | pos: [ 5.19850204 10.04478044] | goal: [ 7.09047511 10.05128897]
t: 783 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.21959034e-04]
maze | pos: [ 5.19850204 10.04484706] | goal: [ 7.09047511 10.05128897]
t: 784 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.19687249e-04]
maze | pos: [ 5.19850204 10.044913  ] | goal: [ 7.09047511 10.05128897]
t: 785 | r: 0.00 |  R: 0.00 | score: -0.0251 | ref_max_score: 273.99 | ref_min_score: 6.7[ 1.89197305e+00 -2.17438716e-04]
maze | pos: [ 5.19850204 10.0449782

In [10]:
scorelist

[(442.0, 1.6285682217815856)]

In [18]:
print("logs/pretrained/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs/pretrained/maze2d-large-v1


(463.0, 24.306377763870945, 170.71345729357625, 9.093635288963647)

In [12]:
print("logs/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs/maze2d-large-v1


(459.8, 9.947864092356712, 169.51625575217926, 3.721749445305367)

In [23]:
print("logs_2/maze2d-large-v1")
np.mean([s[0] for s in scorelist]), np.std([s[0] for s in scorelist]), np.mean([s[1]*100 for s in scorelist]), np.std([s[1]*100 for s in scorelist])

logs_2/maze2d-large-v1


(417.0, 78.03845206050669, 153.5036851359946, 29.1961734672104)

In [12]:
np.std([s[1] for s in scorelist])

21.95702474131786

In [ ]:
json_path = join(savepath, 'rollout.json')
json_data = {'score': score, 'step': t, 'return': total_reward, 'term': terminal,
    'epoch_diffusion': diffusion_experiment.epoch}
json.dump(json_data, open(json_path, 'w'), indent=2, sort_keys=True)

In [ ]:
json_data

{'score': 1.6210857121478544,
 'step': 799,
 'return': 440.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [ ]:
json_data

{'score': 1.3367503460660706,
 'step': 799,
 'return': 364.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [ ]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [ ]:
json_data

{'score': 1.6173444573309887,
 'step': 799,
 'return': 439.0,
 'term': False,
 'epoch_diffusion': 1960000}

In [28]:
100*(439.0-env.ref_min_score)/(env.ref_max_score - env.ref_min_score)

161.73444573309888

In [ ]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [ ]:
print('Ref min score:', env.ref_min_score, 'Ref max score:', env.ref_max_score)

Ref min score: 6.7 Ref max score: 273.99


In [15]:
import json
import numpy as np

In [20]:
metrics = {}

In [21]:
datasets = ['maze2d-large-v1', 'maze2d-medium-v1', 'maze2d-umaze-v1']

In [22]:
for dataset in datasets:
    json_path = f'/root/diffuser/logs/pretrained/{dataset}/rollouts/scorelist.json'
    with open(json_path, 'r') as f:
        data = json.load(f)
    scores = np.array(data['scores'])
    metrics.update({dataset: {'mean': np.mean(scores), 'std': np.std(scores)}})


In [24]:
with open('/root/diffuser/logs/pretrained/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

In [4]:
dataset = 'maze2d-large-v1'
json_path = f'/root/diffuser/logs_2/{dataset}/rollouts/scorelist.json'
with open(json_path, 'r') as f:
    data = json.load(f)
scores = np.array(data['scores'])

FileNotFoundError: [Errno 2] No such file or directory: '/root/diffuser/logs_2/maze2d-large-v1/rollouts/scorelist.json'

In [5]:
scores

NameError: name 'scores' is not defined

In [6]:
np.mean(scores), np.std(scores)

NameError: name 'np' is not defined

In [10]:
import pickle

# Specify the path to the pickle file
pkl_file_path = "logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl"

# Load the pickle file
with open(pkl_file_path, "rb") as f:
    data = pickle.load(f)

# Now you can use the loaded data
print(data)


[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False



In [11]:
dataset_config = utils.Config(
    'datasets.GoalDataset',
    savepath='logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl',
    env='maze2d-large-v1',
    horizon=384,
    normalizer='LimitsNormalizer',
    preprocess_fns=['maze2d_set_terminals'],
    use_padding=False,
    max_path_length=40000,
)

[ utils/config ] Imported diffuser.datasets:GoalDataset

[utils/config ] Config: <class 'diffuser.datasets.sequence.GoalDataset'>
    env: maze2d-large-v1
    horizon: 384
    max_path_length: 40000
    normalizer: LimitsNormalizer
    preprocess_fns: ['maze2d_set_terminals']
    use_padding: False

[ utils/config ] Saved config to: logs/pretrained/maze2d-large-v1/diffusion/H384_T256/dataset_config.pkl



In [12]:
dataset = dataset_config()

load datafile: 100%|██████████| 8/8 [00:01<00:00,  7.19it/s]


[ utils/preprocessing ] Segmented maze2d-large-v1 | 1061 paths | min length: 67 | max length: 30470
[ datasets/buffer ] Finalized replay buffer | 1062 episodes
[ datasets/buffer ] Fields:
    actions: (1062, 40000, 2)
    infos/goal: (1062, 40000, 2)
    infos/qpos: (1062, 40000, 2)
    infos/qvel: (1062, 40000, 2)
    observations: (1062, 40000, 4)
    rewards: (1062, 40000, 1)
    terminals: (1062, 40000, 1)
    timeouts: (1062, 40000, 1)
    next_observations: (1062, 40000, 4)
    normed_observations: (1062, 40000, 4)
    normed_actions: (1062, 40000, 2)


In [13]:
import torch

In [14]:
def cycle(dl):
    while True:
        for data in dl:
            yield data

In [15]:
dataloader = cycle(torch.utils.data.DataLoader(
            dataset, batch_size=1, num_workers=1, shuffle=True, pin_memory=True
        ))

In [16]:
dataloader

<generator object cycle at 0x7fe6d8a1fcf0>

In [17]:
batch = next(dataloader)

In [18]:
from arrays import batch_to_device, to_np, to_device, apply_dict


In [19]:
batch = batch_to_device(batch)

In [20]:
traj = to_np(batch.trajectories)

In [22]:
traj

array([[[ 0.7690196 ,  0.1735717 ,  0.31130707, -0.71825343,
         -0.13593501,  0.3801254 ],
        [ 0.8850416 , -0.32477522,  0.30976582, -0.7141149 ,
         -0.10056651,  0.3871299 ],
        [ 0.76178956, -0.72983533,  0.30884624, -0.7101445 ,
         -0.05999506,  0.37140775],
        ...,
        [ 0.17779183, -1.        , -0.29482794,  0.0478698 ,
          0.01712537,  0.8765112 ],
        [ 0.37615836, -1.        , -0.29444194,  0.05673027,
          0.02518666,  0.82885325],
        [ 0.01288068, -0.37463444, -0.2937941 ,  0.06508267,
          0.04226851,  0.78130853]]], dtype=float32)

In [21]:
conditions = to_np(batch.conditions[0])[:,None]

In [27]:
conditions

array([[[ 0.870726  ,  0.14610803, -0.34367085,  0.10682023]]],
      dtype=float32)